In [9]:
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import torch
import torch.nn as nn

from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler

In [10]:
# Load dataset
data = pd.read_csv("fraudTest.csv")

# Quick check
data.head()


,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,0,2020-06-21 12:14:25,2291163933867244,fraud_Kirlin and Sons,personal_care,2.86,Jeff,Elliott,M,351 Darlene Green,...,33.9659,-80.9355,333497,Mechanical engineer,1968-03-19,2da90c7d74bd46a0caf3777415b3ebd3,1371816865,33.986391,-81.200714,0
1,1,2020-06-21 12:14:33,3573030041201292,fraud_Sporer-Keebler,personal_care,29.84,Joanne,Williams,F,3638 Marsh Union,...,40.3207,-110.4360,302,"Sales professional, IT",1990-01-17,324cc204407e99f51b0d6ca0055005e7,1371816873,39.450498,-109.960431,0
2,2,2020-06-21 12:14:53,3598215285024754,"fraud_Swaniawski, Nitzsche and Welch",health_fitness,41.28,Ashley,Lopez,F,9333 Valentine Point,...,40.6729,-73.5365,34496,"Librarian, public",1970-10-21,c81755dbbbea9d5c77f094348a7579be,1371816893,40.495810,-74.196111,0
3,3,2020-06-21 12:15:15,3591919803438423,fraud_Haley Group,misc_pos,60.05,Brian,Williams,M,32941 Krystal Mill Apt. 552,...,28.5697,-80.8191,54767,Set designer,1987-07-25,2159175b9efe66dc301f149d3d5abf8c,1371816915,28.812398,-80.883061,0
4,4,2020-06-21 12:15:17,3526826139003047,fraud_Johnston-Casper,travel,3.19,Nathan,Massey,M,5783 Evan Roads Apt. 465,...,44.2529,-85.0170,1126,Furniture designer,1955-07-06,57ff021bd3f328f8738bb535c302a31b,1371816917,44.959148,-85.884734,0


In [11]:
# Drop columns with too many categories or not useful
data = data.drop(columns=[
    "cc_num", "first", "last", "street",
    "trans_num", "trans_date_trans_time",
    "job", "merchant", "zip", "dob"
])


In [12]:
# Label encode gender

gender_le = LabelEncoder()
data['gender'] = gender_le.fit_transform(data['gender'])

# One-hot encode 'category' and 'state'

ohe = OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore')
cat_features = ['category', 'state']
ohe_encoded = ohe.fit_transform(data[cat_features])

# Convert back to DataFrame with proper column names

ohe_df = pd.DataFrame(ohe_encoded,
                      columns=ohe.get_feature_names_out(cat_features),
                      index=data.index)

# Concatenate back to dataset and drop originals

df = pd.concat([data.drop(columns=cat_features), ohe_df],
               axis = 1)
df.head()

,Unnamed: 0,amt,gender,city,lat,long,city_pop,unix_time,merch_lat,merch_long,...,state_SD,state_TN,state_TX,state_UT,state_VA,state_VT,state_WA,state_WI,state_WV,state_WY
0,0,2.86,1,Columbia,33.9659,-80.9355,333497,1371816865,33.986391,-81.200714,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,29.84,0,Altonah,40.3207,-110.4360,302,1371816873,39.450498,-109.960431,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,41.28,0,Bellmore,40.6729,-73.5365,34496,1371816893,40.495810,-74.196111,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,60.05,1,Titusville,28.5697,-80.8191,54767,1371816915,28.812398,-80.883061,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,3.19,1,Falmouth,44.2529,-85.0170,1126,1371816917,44.959148,-85.884734,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
